In [31]:
import torch
import torchvision
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms

In [ ]:
df_train = pd.read_csv("/content/sign_mnist_train.csv")
df_test = pd.read_csv("/content/sign_mnist_test.csv")

In [ ]:
# Separate labels and pixel values
labels_train = df_train.iloc[:, 0].values
pixels_train = df_train.iloc[:, 1:].values
print(f"labels = {labels_train}\npixels = {pixels_train}")

labels = [ 3  6  2 ... 18 17 23]
pixels = [[107 118 127 ... 204 203 202]
 [155 157 156 ... 103 135 149]
 [187 188 188 ... 195 194 195]
 ...
 [174 174 174 ... 202 200 200]
 [177 181 184 ...  64  87  93]
 [179 180 180 ... 205 209 215]]


In [ ]:
# Separate labels and pixel values
labels_test = df_test.iloc[:, 0].values
pixels_test = df_test.iloc[:, 1:].values
print(f"labels = {labels_test}\npixels = {pixels_test}")

labels = [ 6  5 10 ...  2  4  2]
pixels = [[149 149 150 ... 112 120 107]
 [126 128 131 ... 184 182 180]
 [ 85  88  92 ... 225 224 222]
 ...
 [190 191 190 ... 211 209 208]
 [201 205 208 ...  67  70  63]
 [173 174 173 ... 195 193 192]]


In [ ]:
class AlexNet(nn.Module):
  def __init__(self, num_classes, dropout=0.5):
      super(AlexNet, self).__init__()
      self.features = nn.Sequential(
          nn.Conv2d(1, 64, 5, stride=1, padding=2),  # Adjusted kernel size, stride, and padding
          nn.ReLU(inplace=True),
          nn.MaxPool2d(kernel_size=2, stride=2),  # Adjusted kernel size and stride
          nn.Conv2d(64, 192, 5, stride=1, padding=2),
          nn.ReLU(inplace=True),
          nn.MaxPool2d(kernel_size=2, stride=2),
          nn.Conv2d(192, 384, kernel_size=3, stride=1, padding=1),
          nn.ReLU(inplace=True),
          nn.Conv2d(384, 256, kernel_size=3, stride=1, padding=1),
          nn.ReLU(inplace=True),
          nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
          nn.ReLU(inplace=True),
          nn.MaxPool2d(kernel_size=2, stride=2)  # Adjusted kernel size and stride
      )
      self.classifier = nn.Sequential(
          nn.Dropout(p=dropout),
          nn.Linear(256 * 3 * 3, 4096),  # Adjusted dimensions based on feature map size
          nn.ReLU(inplace=True),
          nn.Dropout(p=dropout),
          nn.Linear(4096, 4096),
          nn.ReLU(inplace=True),
          nn.Linear(4096, num_classes),  # Adjusted to match the number of classes
          nn.Softmax(dim=1)
      )

  def forward(self, x):
      x = self.features(x)
      x = torch.flatten(x, start_dim=1)  # Flatten the tensor
      x = self.classifier(x)
      return x

In [ ]:
NUM_CLASSES = df_train.iloc[:, 0].nunique()
print(f'Number of unique labels: {NUM_CLASSES}')

Number of unique labels: 24


In [32]:
# Define your device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = AlexNet(num_classes=NUM_CLASSES).to(device)
epochs = 500 # number of single passes on the network
loss_fnPrecents = nn.CrossEntropyLoss(ignore_index=24)
optimizer = torch.optim.SGD(model.parameters(), lr=1e-2)

In [33]:
class CustomDataset(Dataset):
    def __init__(self, pixels, labels, transform=None):
        self.pixels = pixels
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        image = self.pixels[idx].reshape(28, 28).astype('float32')
        label = self.labels[idx].astype('int64')  # Ensure labels are of type int64

        if self.transform:
            image = self.transform(image)

        return image, label

In [34]:
transform = transforms.Compose([
    transforms.ToTensor(),
])
train_dataset = CustomDataset(pixels_train, labels_train, transform=transform)
train_dataloader = DataLoader(train_dataset, batch_size=100)

In [35]:
# Training loop
def train_model(model, dataloader, epochs, optimizer, loss_fn, device):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)

            # Forward pass
            outputs = model(images)
            loss = loss_fn(outputs, labels)

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f"Epoch [{epoch+1}/{epochs}], Loss: {running_loss/len(dataloader):.4f}")

def evaluate_model(model, dataloader, device):
    model.eval()  # Set the model to evaluation mode
    total, correct = 0, 0
    with torch.no_grad():
        for data, targets in dataloader:
            data, targets = data.to(device), targets.to(device)  # Ensure data and targets are on the correct device
            outputs = model(data)
            _, predicted = torch.max(outputs.data, 1)
            total += targets.size(0)
            correct += (predicted == targets).sum().item()

    accuracy = 100 * correct / total  # Calculate accuracy as a percentage
    print(f"Total Correct: {correct}")
    print(f"Total Samples: {total}")
    print(f"Accuracy: {accuracy}%")

In [36]:
train_model(model=model,
            dataloader=train_dataloader,
            epochs=epochs,
            optimizer=optimizer,
            loss_fn=loss_fnPrecents,
            device=device)

Epoch [1/500], Loss: 3.1747
Epoch [2/500], Loss: 3.1620
Epoch [3/500], Loss: 3.1155
Epoch [4/500], Loss: 3.0590
Epoch [5/500], Loss: 2.9915
Epoch [6/500], Loss: 2.9474
Epoch [7/500], Loss: 2.9249
Epoch [8/500], Loss: 2.9097
Epoch [9/500], Loss: 2.8998
Epoch [10/500], Loss: 2.8696
Epoch [11/500], Loss: 2.8544
Epoch [12/500], Loss: 2.8480
Epoch [13/500], Loss: 2.8438
Epoch [14/500], Loss: 2.8419
Epoch [15/500], Loss: 2.8187
Epoch [16/500], Loss: 2.7982
Epoch [17/500], Loss: 2.7715
Epoch [18/500], Loss: 2.7628
Epoch [19/500], Loss: 2.7560
Epoch [20/500], Loss: 2.7540
Epoch [21/500], Loss: 2.7519
Epoch [22/500], Loss: 2.7507
Epoch [23/500], Loss: 2.7497
Epoch [24/500], Loss: 2.7488
Epoch [25/500], Loss: 2.7479
Epoch [26/500], Loss: 2.7475
Epoch [27/500], Loss: 2.7466
Epoch [28/500], Loss: 2.7213
Epoch [29/500], Loss: 2.7061
Epoch [30/500], Loss: 2.7031
Epoch [31/500], Loss: 2.7018
Epoch [32/500], Loss: 2.7012
Epoch [33/500], Loss: 2.7006
Epoch [34/500], Loss: 2.7005
Epoch [35/500], Loss: 2

In [37]:
val_dataset = CustomDataset(pixels_test, labels_test, transform=transform)
val_dataloader = DataLoader(val_dataset, batch_size=100)

In [38]:
evaluate_model(
    model=model,
    dataloader=train_dataloader,
    device=device
)

Total Correct: 26337
Total Samples: 27455
Accuracy: 95.92788198870879%


In [39]:
evaluate_model(
    model=model,
    dataloader=val_dataloader,
    device=device
)

Total Correct: 6655
Total Samples: 7172
Accuracy: 92.79141104294479%


In [51]:
for name, param in model.named_parameters():
    print(f"{name} - parameter:\n", param.data)

features.0.weight - parameter:
 tensor([[[[ 3.3887e-02,  1.0412e-01, -2.0227e-01,  6.6796e-02,  1.8589e-01],
          [-1.9303e-02,  1.3635e-01,  1.7168e-01,  1.5087e-01, -1.1247e-01],
          [-1.0118e-01, -1.4005e-01, -1.1336e-01, -1.6930e-01,  1.7974e-01],
          [-1.7295e-01,  1.4025e-01,  9.6501e-02, -3.4034e-02,  8.5212e-02],
          [ 1.3568e-01,  1.0634e-01,  1.0532e-01, -4.2140e-02,  1.8517e-01]]],


        [[[-1.6751e-01,  1.3910e-01, -1.0123e-01,  7.2488e-03, -1.3555e-01],
          [ 8.9740e-02,  7.0290e-02, -1.0404e-01,  1.0112e-01, -1.6532e-01],
          [ 4.5242e-02,  1.2659e-01, -1.0964e-01, -3.3610e-02,  2.9541e-02],
          [-1.5014e-02, -1.1339e-01,  1.2464e-01, -1.5295e-01,  1.2082e-01],
          [ 1.0550e-01, -3.8845e-02,  1.6478e-01, -1.3668e-01, -1.2532e-01]]],


        [[[-1.2475e-01,  1.0751e-02, -1.1704e-01,  1.3578e-01,  1.7022e-01],
          [-7.1269e-02, -1.7054e-01,  8.3490e-02,  2.1340e-01, -1.0352e-01],
          [ 5.8848e-02,  1.2889e-01,

In [50]:
# Assuming 'model' is your trained model
torch.save(model.state_dict(), 'model_weights.pth')  # Saves only the weights